# Data Statistics

In [3]:
%load_ext autoreload
%autoreload 2

import plotly.express as px

from htc.settings import settings
from htc.settings_seg import settings_seg  # noqa: F401
from htc.utils.helper_functions import basic_statistics
from htc.utils.LabelMapping import LabelMapping
from htc.utils.sqldf import sqldf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
sorted_labels = sorted(LabelMapping.from_data_dir(settings.data_dirs.semantic).label_names(include_invalid=True))
df = basic_statistics("2021_02_05_Tivita_multiorgan_semantic", "pigs_semantic-only_5foldsV2.json")
df.head()

,image_name,subject_name,timestamp,label_name,n_pixels,set_type
0,P041#2019_12_14_12_00_16,P041,2019_12_14_12_00_16,bladder,10594,train
1,P041#2019_12_14_12_00_16,P041,2019_12_14_12_00_16,blue_cloth,158624,train
2,P041#2019_12_14_12_00_16,P041,2019_12_14_12_00_16,colon,67779,train
3,P041#2019_12_14_12_00_16,P041,2019_12_14_12_00_16,fat,4251,train
4,P041#2019_12_14_12_00_16,P041,2019_12_14_12_00_16,small_bowel,65634,train


## Label Distribution

In [5]:
df_labels = sqldf(
    """
    SELECT label_name, subject_name, COUNT(*) AS n_images, SUM(n_pixels) AS n_pixels
    FROM df
    WHERE label_name NOT IN ('unlabeled', 'overlap')
    GROUP BY label_name, subject_name
"""
)

fig = px.bar(df_labels, x="label_name", y="n_images", color="subject_name")
fig.update_layout(title_text="Number of images per label", title_x=0.5)
fig.layout.xaxis.categoryorder = "array"
fig.layout.xaxis.categoryarray = sorted_labels
fig.layout.height = 500
fig.layout.width = 2000
fig.update_yaxes(matches=None, showticklabels=True)
fig.show()

In [6]:
fig = px.bar(df_labels, x="label_name", y="n_pixels", color="subject_name")
fig.update_layout(title_text="Number of pixels per label", title_x=0.5)
fig.layout.xaxis.categoryorder = "array"
fig.layout.xaxis.categoryarray = sorted_labels
fig.layout.height = 500
fig.layout.width = 2000
fig.update_yaxes(matches=None, showticklabels=True)
fig.show()

In [7]:
df_images = sqldf(
    """
    SELECT label_name, subject_name, set_type, COUNT(*) AS n_images, SUM(n_pixels) AS n_pixels
    FROM df
    WHERE label_name NOT IN ('unlabeled', 'overlap')
    GROUP BY label_name, subject_name, set_type
"""
)

fig = px.bar(df_images, x="label_name", y="n_images", color="subject_name", facet_col="set_type")
fig.update_layout(title_text="Number of images per label", title_x=0.5)
fig.layout.xaxis.categoryorder = "array"
fig.layout.xaxis.categoryarray = sorted_labels
fig.layout.height = 500
fig.layout.width = 2000
fig.update_yaxes(matches=None, showticklabels=True)
fig.show()

In [8]:
fig = px.bar(df_images, x="label_name", y="n_pixels", color="subject_name", facet_col="set_type")
fig.update_layout(title_text="Number of images per label", title_x=0.5)
fig.layout.xaxis.categoryorder = "array"
fig.layout.xaxis.categoryarray = sorted_labels
fig.layout.height = 500
fig.layout.width = 2000
fig.update_yaxes(matches=None, showticklabels=True)
fig.show()

## Label Counts

In [9]:
df_label_count = sqldf(
    """
    SELECT n_labels, COUNT(*) n_images
    FROM (
        SELECT timestamp, COUNT(label_name) AS n_labels
        FROM df
        WHERE label_name NOT IN ('unlabeled', 'overlap')
        GROUP BY timestamp
    )
    GROUP BY n_labels

    UNION

    SELECT 0, COUNT(*)
    FROM (
        SELECT timestamp, COUNT(label_name) AS n_labels
        FROM df
        GROUP BY timestamp
        HAVING n_labels = 1 AND label_name = 'unlabeled'
    )
"""
)

fig = px.bar(df_label_count, x="n_labels", y="n_images")
fig.update_layout(title_x=0.5, title_text="How many images do we have which contain n different labels?")
fig.show()

In [10]:
# If only 1 label is present in the image, it must be unlabeled
df_1label = sqldf(
    """
    SELECT timestamp
    FROM df
    GROUP BY timestamp
    HAVING COUNT(label_name) = 1
"""
)
df_unlabeled = sqldf(
    """
    SELECT timestamp
    FROM df
    GROUP BY timestamp
    HAVING COUNT(label_name) = 1 AND label_name = 'unlabeled'
"""
)
all(df_1label == df_unlabeled)

True

## Additional Test Set Pig

In [11]:
new_labels = ["muscle", "kidney", "major_vein", "kidney_with_Gerotas_fascia"]
df_new = df.query("label_name in @new_labels")
df_new_pigs = sqldf(
    """
    SELECT label_name, GROUP_CONCAT(subject_name, ',') AS pigs
    FROM (
        SELECT DISTINCT label_name, subject_name
        FROM df_new
    )
    GROUP BY label_name
"""
)

df_new_pigs

,label_name,pigs
0,kidney,"P058,P059,P061,P062,P072"
1,kidney_with_Gerotas_fascia,"P058,P059,P060,P062,P072"
2,muscle,"P044,P060,P061,P062,P070,P072"
3,major_vein,"P058,P059,P060,P062,P072"


In [12]:
sqldf(
    """
    SELECT subject_name, label_name, COUNT(*) AS n_images
    FROM df_new
    WHERE subject_name IN ('P062', 'P072')
    GROUP BY subject_name, label_name
"""
)

,subject_name,label_name,n_images
0,P062,kidney,17
1,P062,kidney_with_Gerotas_fascia,10
2,P062,muscle,12
3,P062,major_vein,10
4,P072,kidney,9
5,P072,kidney_with_Gerotas_fascia,1
6,P072,muscle,6
7,P072,major_vein,8


In [13]:
df_organs = df.query("label_name in @settings_seg.labels")
sqldf(
    """
    SELECT label_name, COUNT(DISTINCT subject_name) AS n_pigs
    FROM df_organs
    GROUP BY label_name
    ORDER BY n_pigs
"""
)

,label_name,n_pigs
0,kidney,5
1,kidney_with_Gerotas_fascia,5
2,major_vein,5
3,muscle,6
4,heart,8
5,lung,8
6,pancreas,10
7,background,11
8,gallbladder,13
9,bladder,14
